In [155]:

#@title Imports
import cv2
from google.colab.patches import cv2_imshow
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
import math


In [156]:
#@title Importing the image and the world coordinates (getting them from file to array) and the imaged coordinates


drive.mount("/content/drive/")  # Mounting Google Drive to access files

coordinates_path = '/content/drive/MyDrive/CV-project2/coords.txt'  # Path to the coordinates file

# Read the world coordinate file
with open(coordinates_path) as f:
    world_coordinates = f.read()

lines = world_coordinates.split('\n')  # Split the file content into lines

real_world = []  # List to store the world coordinates

# Iterate over each line
for line in lines:
    coordinates = line.split()  # Split the line into individual coordinates

    # Convert the coordinates to float and add them to the list
    coordinates = [float(coord) for coord in coordinates]
    real_world.append(coordinates)

real_world = real_world[:-1]  # Remove the last element (empty line)
real_world = np.array(real_world)  # Convert the list to a NumPy array
real_world_no_homogen = np.delete(real_world, -1, axis=1)  # Remove the last column (assumed to be homogeneous coordinates)
# print(real_world)
# print(real_world_no_homogen)

# print(real_world)  # Print the resulting world coordinates array

manually_figured_points = np.array([[286, 408, 1],
                                    [414, 265, 1],
                                    [482, 272, 1],
                                    [607, 60, 1],
                                    [750, 213, 1],
                                    [574, 483, 1],
                                    [410, 56, 1],
                                    [679, 336, 1],
                                    [276, 243, 1],
                                    [734, 392, 1]])



house2_10points = np.array([[286, 443, 1],
                           [342, 286, 1],
                           [375, 286, 1],
                           [689, 70, 1],
                           [807, 246, 1],
                           [387, 536, 1],
                           [341, 55, 1],
                           [661, 389, 1],
                           [282, 266, 1],
                           [790, 478, 1]])


# manually_figured_points_no_homogen = np.delete(manually_figured_points, -1, axis=1)
# print(np.shape(manually_figured_points))  # Print the shape of the manually figured points array





Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [157]:
#@title Find the projection matrix
from numpy.linalg import matrix_rank
from scipy.optimize import minimize
import numpy as np

x = manually_figured_points
X = real_world


A = [] # Initialize the list A

for i in range(len(manually_figured_points)):

    A.append([X[i][0], X[i][1], X[i][2], 1, 0, 0, 0, 0, *(-x[i][0]*X[i])])
    A.append([0, 0, 0, 0, X[i][0], X[i][1], X[i][2], 1, *(-x[i][1]*X[i])])

A = np.array(A)
# Singular Value Decomposition (SVD) of A
_, _, V = np.linalg.svd(A)

# Smallest singular vector corresponding to the smallest singular value
P_house_1 = V[-1].reshape(3,4)



print(P_house_1)


[[-1.35985036e-01 -2.05287976e-01  1.24204300e-02 -7.42875968e-01]
 [ 7.32370218e-03 -1.07456747e-02  2.35700948e-01 -5.75864392e-01]
 [ 1.02347989e-04 -1.38353653e-04  2.54937711e-05 -1.44034056e-03]]


In [158]:
print(np.shape(V))

(12, 12)


In [159]:
x = house2_10points
X = real_world


A = [] # Initialize the matrix A

for i in range(len(manually_figured_points)):

    # Form the rows of A based on the equation X_hat_i * p = 0
    A.append([X[i][0], X[i][1], X[i][2], 1, 0, 0, 0, 0, *(-x[i][0]*X[i])])
    A.append([0, 0, 0, 0, X[i][0], X[i][1], X[i][2], 1, *(-x[i][1]*X[i])])

A = np.array(A)
# Singular Value Decomposition (SVD) of A
_, _, V = np.linalg.svd(A)

# Smallest singular vector corresponding to the smallest singular value
P_house_2 = V[-1].reshape(3,4)

print(P_house_2)



[[-2.23612445e-02 -2.65920394e-01  3.93050773e-03 -6.97004619e-01]
 [ 3.45279752e-02 -1.53916924e-02  2.53249123e-01 -6.14320275e-01]
 [ 1.71136615e-04 -7.85605022e-05  9.38318696e-06 -1.33841542e-03]]


In [160]:
pi = np.array([[0,0,1], [0,1,0], [1,0,0]])
mpi_house1 = P_house_1[:, :3].T@pi
mpi_house2 = P_house_2[:, :3].T@pi



def QR_Decomposition(A):
    n, m = A.shape # get the shape of A

    Q = np.empty((n, n)) # initialize matrix Q
    u = np.empty((n, n)) # initialize matrix u

    u[:, 0] = A[:, 0]
    Q[:, 0] = u[:, 0] / np.linalg.norm(u[:, 0])

    for i in range(1, n):

        u[:, i] = A[:, i]
        for j in range(i):
            u[:, i] -= (A[:, i] @ Q[:, j]) * Q[:, j] # get each u vector

        Q[:, i] = u[:, i] / np.linalg.norm(u[:, i]) # compute each e vector

    R = np.zeros((n, m))
    for i in range(n):
        for j in range(i, m):
            R[i, j] = A[:, j] @ Q[:, i]

    return Q, R

Q, R = QR_Decomposition(mpi_house1)
Q2, R2 = QR_Decomposition(mpi_house2)


K = pi@R.T@pi
R = pi@Q.T
print(f'Rotation matrix for house 1:\n{R}')
print(f'Kalibration matrix for house 1:\n{K}')
print("\n")
K2 = pi@R2.T@pi
R2 = pi@Q2.T
print(f'Rotation matrix for house 2:\n{R2}')
print(f'Kalibration matrix for house 2:\n{K2}')
# K = K / K[-1, -1]
# print(K)
# K2 = K2 / K2[-1, -1]
# print(K2)

Rotation matrix for house 1:
[[-0.80374416 -0.59497117 -0.00215096]
 [-0.0888966   0.11651386  0.98920267]
 [ 0.58829645 -0.79525708  0.14653825]]
Kalibration matrix for house 1:
[[2.31410891e-01 4.56035772e-04 8.50772707e-02]
 [0.00000000e+00 2.31252934e-01 4.73932861e-02]
 [0.00000000e+00 0.00000000e+00 1.73973494e-04]]


Rotation matrix for house 2:
[[-0.41726842 -0.90878178  0.00165744]
 [-0.04453716  0.02227084  0.99875946]
 [ 0.90769131 -0.41667696  0.04976748]]
Kalibration matrix for house 2:
[[ 2.51000764e-01 -1.00073124e-03  9.07014062e-02]
 [ 0.00000000e+00  2.51054392e-01  5.03576759e-02]
 [ 0.00000000e+00  0.00000000e+00  1.88540547e-04]]


In [161]:
#@title Find the translation matrix
last_column = P_house_1[:, -1]
translation_1 = -np.linalg.inv(P_house_1[:,:3]) @ last_column
print(f'C_tilde for house 1:\n {translation_1}')


last_column1 = P_house_2[:, -1]
translation_2 = -np.linalg.inv(P_house_2[:,:3]) @ last_column1
print(f'C_tilde for house 2:\n {translation_2}')

C_tilde for house 1:
 [ 4.66750445 -6.58963996  1.99774696]
C_tilde for house 2:
 [ 6.30794468 -3.13120773  1.37542483]


In [162]:
p1 = P_house_1[:,0]
p2 = P_house_1[:,1]
p3 = P_house_1[:,2]
p4 = P_house_1[:,3]


print(np.allclose(K@R, P_house_1[:, :3]))
print(np.allclose(-P_house_1[:, :3]@translation_1, p4))

True
True


In [163]:
p1 = P_house_2[:,0]
p2 = P_house_2[:,1]
p3 = P_house_2[:,2]
p4 = P_house_2[:,3]


print(np.allclose(K2@R2, P_house_2[:, :3]))

print(np.allclose(-P_house_2[:, :3]@translation_2, p4))

True
True
